In [2]:
# Regular data science libraries
import numpy as np
import pandas as pd
import json
import os
import pickle
import random


# DNN libraries
import torch
import torch.nn as nn
import torch.nn.functional as F


# Import utility functions from other files
# from utils import generate_model, dp, generate_offline_data
# from env import Agent, Environment


# Plotting libraries
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import scienceplots
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
plt.style.use('ieee')


# Clustering libraries
from sklearn.decomposition import PCA
from pyclustering.cluster import cluster_visualizer
from pyclustering.cluster.xmeans import xmeans
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.utils import read_sample
from pyclustering.samples.definitions import SIMPLE_SAMPLES
from scipy.stats import wasserstein_distance

# Set the seeds
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)


/root/miniconda3/envs/FACT-AI/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import gym
import ale_py
import d4rl_atari
print(ale_py.__version__)
print(gym.__version__)
env = gym.make('seaquest-mixed-v4') # -v{0, 1, 2, 3, 4} for datasets with the other random seeds
# env = gym.make('ALE/Breakout-v5')

# interaction with its environment through dopamine-style Atari wrapper
observation = env.reset() # observation.shape == (84, 84)

observation, reward, terminal, truncated, info = env.step(env.action_space.sample())

# dataset will be automatically downloaded into ~/.d4rl/datasets/[GAME]/[INDEX]/[EPOCH]
dataset = env.get_dataset()


0.8.1
0.26.2


/root/miniconda3/envs/FACT-AI/lib/python3.7/site-packages/gym/utils/passive_env_checker.py:32: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (84, 84)
  "A Box observation space has an unconventional shape (neither an image, nor a 1D vector). "


FileNotFoundError: [Errno 2] No such file or directory: 'gsutil': 'gsutil'

In [4]:
# print(dataset['observations'])
print(dataset['observations'].shape) # observation data in (1000000, 1, 84, 84)
print(type(dataset['observations']))

NameError: name 'dataset' is not defined

In [5]:
print(np.unique(dataset['actions'])) # action data in (1000000,)
print(dataset['actions'].shape) # observation data in (1000000, )
print(type(dataset['actions']))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
(1000000,)
<class 'numpy.ndarray'>


In [6]:
print(np.unique(dataset['rewards'])) # reward data in (1000000,)
print(dataset['rewards'].shape) # observation data in (1000000, )
print(type(dataset['rewards']))

[0. 1.]
(1000000,)
<class 'numpy.ndarray'>


In [7]:
print(np.unique(dataset['terminals'])) # terminal flags in (1000000,)
print(dataset['terminals'].shape) # observation data in (1000000, )
print(type(dataset['terminals']))

[0 1]
(1000000,)
<class 'numpy.ndarray'>


In [1]:
from decision_transformer_atari import GPTConfig, GPT

In [2]:
import torch
print(torch.__version__)

1.13.1


In [6]:
vocab_size = 18 #4
block_size = 90
model_type = "reward_conditioned"
timesteps = 2719 #2654

mconf = GPTConfig(
    vocab_size,
    block_size,
    n_layer=6,
    n_head=8,
    n_embd=128,
    model_type=model_type,
    max_timestep=timesteps,
)
model = GPT(mconf)

# map_location=torch.device('cpu')
model.load_pretrained("checkpoints/Seaquest_123.pth")

# checkpoint_path = "checkpoints/Breakout_123.pth"  # or Pong, Qbert, Seaquestcheckpoint = torch.load(checkpoint_path)
# model.load_state_dict(checkpoint)


In [7]:
model

GPT(
  (tok_emb): Embedding(18, 128)
  (drop): Dropout(p=0.1, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (attn): CausalSelfAttention(
        (key): Linear(in_features=128, out_features=128, bias=True)
        (query): Linear(in_features=128, out_features=128, bias=True)
        (value): Linear(in_features=128, out_features=128, bias=True)
        (attn_drop): Dropout(p=0.1, inplace=False)
        (resid_drop): Dropout(p=0.1, inplace=False)
        (proj): Linear(in_features=128, out_features=128, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): GELU()
        (2): Linear(in_features=512, out_features=128, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
    (1): Block(
      (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ln